In [59]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [60]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [61]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [62]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [63]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])
market_scaled[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [64]:
# Create a DataFrame with the scaled data
df_market_scaled = pd.DataFrame(market_scaled, columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'])


# Copy the crypto names from the original data
df_market_scaled["coin_id"] = df_market_data.index
# Set the coinid column as index
df_market_scaled.set_index('coin_id',inplace=True)

# Display sample data
df_market_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [65]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [76]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_scaled)
    inertia.append(k_model.inertia_)

c:\Users\m1365\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [77]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_curve_data)
df_elbow

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.405923
5,6,52.933559
6,7,47.983124
7,8,37.288187
8,9,33.061685
9,10,28.779752


In [133]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Original Elbow Curve", 
    xticks=k
)
elbow_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 
Based on the provided Elbow Curve, the optimal value for \( k \) is often identified at the "elbow" of the curve, where the rate of inertia decrease slows. Observing the curve, there is a prominent decrease in inertia when moving from \( k=1 \) to \( k=3 \). Beyond \( k=3 \), the rate of reduction diminishes notably. Hence, the best value for \( k \) would likely be \( k=4 \), as this is the point where the addition of more clusters doesn't yield a significantly improved fit.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [79]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [82]:
# Fit the K-Means model using the scaled data
model.fit(df_market_scaled)

KMeans(n_clusters=4, random_state=0)

In [84]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_segs = model.predict(df_market_scaled)

# Print the resulting array of cluster values.
print(market_segs)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [87]:
# Create a copy of the DataFrame
df_market_predictions = df_market_scaled.copy()

In [88]:
# Add a new column to the DataFrame with the predicted clusters
df_market_predictions["predict_column"] = market_segs
df_market_predictions['hover_cols'] = df_market_data.index
# Display sample data
df_market_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predict_column,hover_cols
coin_id,,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0,bitcoin
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0,ethereum
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3,tether
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,3,ripple
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0,bitcoin-cash


In [90]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_plot = df_market_predictions.hvplot.scatter(x = "price_change_percentage_24h", 
                                 y = "price_change_percentage_7d",
                                 by = "predict_column",
                                 title = "market_plot"
                            )
market_plot

:NdOverlay   [predict_column]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

---

### Optimize Clusters with Principal Component Analysis.

In [92]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [94]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca = pca.fit_transform(df_market_scaled)
# View the first five rows of the DataFrame. 
market_pca[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [103]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
array_variance_ratio = pca.explained_variance_ratio_
array_variance_ratio,"and Sum :" , array_variance_ratio.sum()




(array([0.3719856 , 0.34700813, 0.17603793]), 'and Sum :', 0.8950316570309841)

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is the sum of the variances of each of these components. If, for instance, the variances for the first, second, and third principal components are 50%, 30%, and 10% respectively, then the total explained variance for the three components would be \(50% + 30% + 10% = 90%\). This means that these three components together capture 90% of the total variance in the dataset.

In [104]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
df_market_pca = pd.DataFrame(market_pca,columns=["PC1","PC2","PC3"])

# Copy the crypto names from the original data
df_market_pca["coin_id"] = df_market_scaled.index

# Set the coinid column as index
df_market_pca.set_index("coin_id",inplace=True)

# Display sample data
df_market_pca.head(5)

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [105]:
# Create a list with the number of k-values from 1 to 11
pca_k = list(range(1, 11))

In [109]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in pca_k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_pca)
    pca_inertia.append(k_model.inertia_)

c:\Users\m1365\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [110]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {"k": pca_k, "inertia": pca_inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow_data)
df_pca_elbow

,k,inertia
0,1,256.874086
1,2,165.901994
2,3,93.774626
3,4,49.665497
4,5,38.352251
5,6,27.618972
6,7,21.134056
7,8,17.437664
8,9,13.742792
9,10,10.484890


In [134]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca_plot = df_pca_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve with PCA Method", 
    xticks=pca_k
)
elbow_pca_plot

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**
The best value for `k` when using the PCA data is typically determined by evaluating the cumulative explained variance versus the number of principal components or through a specific application need, such as clustering or classification. One common method is the elbow method, where the variance explained is plotted against the number of principal components, and the "elbow" of the curve indicates an optimal value for `k`. Without specific data, I cannot provide an exact value.

* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 
  It might. The best `k` value for PCA is concerned with dimensionality reduction and maximizing variance capture, whereas the best `k` in other contexts, like k-means clustering, might be determined by cluster cohesion and separation. When comparing PCA-transformed data to the original data, the best value for `k` might differ depending on the nature of the data and the specific task at hand.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [117]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=4)

In [118]:
# Fit the K-Means model using the PCA data
model.fit(df_market_pca)

KMeans(n_clusters=4, random_state=4)

In [119]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_pca_4 = model.predict(df_market_pca)
# Print the resulting array of cluster values.
k_pca_4

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 0, 1, 1, 2, 1, 1, 1, 1])

In [120]:
# Create a copy of the DataFrame with the PCA data
market_pca_predict_df = df_market_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
market_pca_predict_df['hover_cols'] = k_pca_4
k_pca_4

# Display sample data
market_pca_predict_df.head(5)

,PC1,PC2,PC3,hover_cols
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,1
ripple,-0.471835,-0.222660,-0.479053,1
bitcoin-cash,-1.157800,2.041209,1.859715,0


In [128]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_pca_plot = market_pca_predict_df.hvplot.scatter(x = "PC1", 
                                     y = "PC2",
                                     by = "hover_cols",
                                     title = 'market_pca_plot'
                                   )
market_pca_plot

:NdOverlay   [hover_cols]
   :Scatter   [PC1]   (PC2)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [135]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_pca_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [130]:
# Composite plot to contrast the clusters
market_plot + market_pca_plot 



:Layout
   .NdOverlay.I  :NdOverlay   [predict_column]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [hover_cols]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Principal Component Analysis (PCA) is a technique employed to reduce the dimensionality of datasets. In the context you provided, PCA was utilized to transform the original dataset by combining its various features to create new PCA-derived variables. One primary reason for employing PCA, especially in this context, is to achieve better visualization, particularly when aiming to discern patterns or clusters in the data.

The effectiveness of PCA in this scenario can be observed when comparing two plots: the "market_plot" and the "market_pca_plot".

In the "market_plot", which represents clustering using K-Means on the original dataset, we notice an overlap between two clusters. This overlap can make it challenging to distinguish between different data points and to draw clear conclusions.

Contrarily, the "market_pca_plot", which showcases the data post-PCA application and subsequent K-Means clustering, presents a clearer distinction with four separate clusters. The division between clusters in this plot is more apparent, and there's no visible overlap. This result signifies the potential of PCA in enhancing the clarity of data visualization and in ensuring more distinct and interpretable clustering outcomes.

In conclusion, PCA can be a potent tool in data preprocessing, especially when the aim is to enhance visualization clarity or when dealing with datasets with many features. Its application in conjunction with clustering algorithms like K-Means can lead to more discernible and meaningful results.